# **Step1 : Install The package**

In [26]:
!pip install crewai
!pip install crewai-tools



📝 **Test Case**: Planning a Last-Minute Budget Trip to Goa

**Scenario**:

You have only 3 days for the trip
Your budget is Rs15000 (including flights,stay, food, transport).

You prefer historical sites over shopping.

You need to find a hotel near public transport.

You want a real-time weather forecast for Goa before finalizing the plan.

# **Step 2 : Import the package**

In [27]:
import os #operating system
from crewai import Agent, Task, Crew, Process, LLM #CrewAI package
from crewai_tools import SerperDevTool              # Seper Tool
import streamlit as st


# **Step 3 : Access The Keys**

In [28]:
from google.colab import userdata
gemini_key = userdata.get('gemini_key')
serper_key = userdata.get('serper_key')
os.environ['GOOGLE_API_KEY'] = gemini_key
os.environ['SERPER_API_KEY'] = serper_key


# **Step 4 : LLM and Tools**

In [29]:
# Initialize a search tool (to fetch real-time travel info)
search_tool = SerperDevTool()

# Define the AI Model
llm = LLM(model="gemini/gemini-1.5-flash", #model
          verbose=True, # i want to see the see statements
          temperature=0.5, #thinking power
          api_key=os.environ["GOOGLE_API_KEY"])

# **Step 5 : Intialize the Agents**

In [30]:
# 🧭 Travel Researcher Agent (Finds historical sites + weather)
researcher = Agent(
    role="Travel Researcher",
    goal="Find historical sites, public transport hotels, and real-time weather for {destination}.",
    verbose=True, # i want to see the statement
    memory=True, #store
    backstory="You are an expert travel researcher, providing up-to-date information about history-focused trips.",
    llm=llm,
    tools=[search_tool],  # Uses live search tool
    allow_delegation=True # Transfer the Task to the
)



# **Step 6 : Agent Task**

In [31]:
# 📝 Travel Research Task
research_task = Task(
    description="Find the best historical sites, weather forecast, and public transport hotels for {destination}.",
    expected_output="A list of top historical sites, a real-time weather update, and 3 hotel options near public transport.",
    tools=[search_tool],
    agent=researcher
)

# 💲 Budget Estimation Task
budget_task = Task(
    description="Find budget flights, hotel options, and daily food/transport costs for {destination}. Ensure total cost stays under {budget}.",
    expected_output="A full cost breakdown (flights, hotel, food, attractions) ensuring a $1500 budget is maintained.",
    tools=[search_tool],
    agent=budget_planner
)

# 📅 Itinerary Planning Task
itinerary_task = Task(
    description="Plan a 3-day itinerary for {destination}, focusing on historical sites, budget constraints, and real-time weather conditions.",
    expected_output="A detailed 3-day plan, considering weather and budget constraints, with transport recommendations.",
    tools=[search_tool],
    agent=itinerary_planner
)


# **Step 7 Kick off the work**

In [32]:
crew = Crew(
    agents=[researcher, budget_planner, itinerary_planner],
    tasks=[research_task, budget_task, itinerary_task],
    process=Process.sequential  # Runs tasks in sequence
)

# 🔥 Run the CrewAI Trip Advisor system for London with a RS.25000 budget
result = crew.kickoff(inputs={'destination': 'Delhi', 'budget': '25000 INR'})
print(result)

# Agent: Travel Researcher
## Task: Find the best historical sites, weather forecast, and public transport hotels for Delhi.


# Agent: Travel Researcher
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"Top historical sites in Delhi, India\"}"
## Tool Output: 
{'searchParameters': {'q': 'Top historical sites in Delhi, India', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'THE 10 BEST New Delhi Sights & Historical Landmarks to Visit (2025)', 'link': 'https://www.tripadvisor.com/Attractions-g304551-Activities-c47-New_Delhi_National_Capital_Territory_of_Delhi.html', 'snippet': "3. Humayun's Tomb · 4. Gurudwara Bangla Sahib · 5. Lotus Temple · 6. India Gate · 7. Red Fort · 8. Hauz Khas Village · 9. Connaught Place · 10.", 'position': 1}, {'title': '12 Famous Historical Places In Delhi- A Cultural Journey', 'link': 'https://www.captureatrip.com/blog/historical-places-in-delhi', 'snippet': "1. Red Fort: It is considered one of the